### I) Loading and Exploring the Dataset

In [62]:
%matplotlib inline
import os
import pandas as pd
import requests
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
characters = pd.read_csv('./data/raw/CMU_movies/character.metadata.tsv', sep='\t', 
                         names=['wiki_movie_id', 'free_movie_id', 'release', 'char_name', 'actor_birth', 'actor_gender', 'actor_height',
                                 'actor_ethnicity', 'actor_name', 'age_at_release', 'free_map_id', 'free_char_id', 'free_actor_id'] )
movies = pd.read_csv('./data/raw/CMU_movies/movie.metadata.tsv', sep='\t', names=['wiki_movie_id', 'free_movie_id', 'movie_name', 'release', 
                                                                   'box_office', 'runtime', 'languages', 'countries', 'genres'])
summaries = df = pd.read_csv('./data/raw/CMU_movies/plot_summaries.txt', delimiter='\t', header=None, names=['movie_id', 'plot_summary'], encoding='utf-8')

In [64]:
print(characters.shape)
print(movies['movie_name'].nunique())
print(summaries.shape)

(450669, 13)
75478
(42303, 2)


In [65]:
characters.head(2)

,wiki_movie_id,free_movie_id,release,char_name,actor_birth,actor_gender,actor_height,actor_ethnicity,actor_name,age_at_release,free_map_id,free_char_id,free_actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.62,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.78,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4


In [66]:
movies.head(2)

,wiki_movie_id,free_movie_id,movie_name,release,box_office,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."


In [67]:
summaries.head(2)

,movie_id,plot_summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...


In [68]:
print(characters.isnull().sum())

wiki_movie_id           0
free_movie_id           0
release              9995
char_name          257875
actor_birth        106145
actor_gender        45609
actor_height       295845
actor_ethnicity    344611
actor_name           1228
age_at_release     158113
free_map_id             0
free_char_id       257865
free_actor_id         815
dtype: int64


In [69]:
print(movies.isnull().sum())

wiki_movie_id        0
free_movie_id        0
movie_name           0
release           6902
box_office       73340
runtime          20450
languages            0
countries            0
genres               0
dtype: int64


In [70]:
print(summaries.isnull().sum())

movie_id        0
plot_summary    0
dtype: int64


### II) Cleaning the Dataset

In [71]:
# Standardize 'release' column to extract the correct year
def extract_year(release_date):
    try:
        # Attempt to convert to datetime and extract the year
        year = pd.to_datetime(release_date, errors='coerce').year
        if year is not pd.NaT:  # Check if the year is valid
            return year
        # If conversion fails, try extracting just the first 4 digits as year
        return int(str(release_date)[:4])
    except (ValueError, TypeError):
        return None  # Return None if extraction fails

# Apply the function to the 'release' column
movies['release'] = movies['release'].apply(extract_year)

movies.head(2)

,wiki_movie_id,free_movie_id,movie_name,release,box_office,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001.0,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000.0,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."


In [72]:
print(movies['movie_name'].nunique())

75478


In [73]:
# We are checking whether an inner join is desirable or if we should use an outer join instead
missing_in_summaries = movies[~movies['wiki_movie_id'].isin(summaries['movie_id'])]
print(f"Movies missing in summaries: {len(missing_in_summaries)}")

missing_in_movies = summaries[~summaries['movie_id'].isin(movies['wiki_movie_id'])]
print(f"Summaries missing in movies: {len(missing_in_movies)}")

missing_in_characters = movies[~movies['wiki_movie_id'].isin(characters['wiki_movie_id'])]
print(f"Movies missing in characters: {len(missing_in_characters)}")


Movies missing in summaries: 39537
Summaries missing in movies: 99
Movies missing in characters: 17411


In [74]:
print(movies['movie_name'].nunique())
print(movies['movie_name'].isnull().sum())
print(summaries.shape)

75478
0
(42303, 2)


In [75]:
# Merge movies and summaries 
full_dataset = movies.merge(summaries, how='outer', left_on='wiki_movie_id', right_on='movie_id')
# Arbitrarily drop one of the 2 columns with same ids for movie
full_dataset.drop(columns=['movie_id'], inplace=True)

# Merge our movies-summary dataset with the characters one
full_dataset = full_dataset.merge(characters, how='outer', on='wiki_movie_id')
full_dataset.head(2)

,wiki_movie_id,free_movie_id_x,movie_name,release_x,box_office,runtime,languages,countries,genres,plot_summary,...,char_name,actor_birth,actor_gender,actor_height,actor_ethnicity,actor_name,age_at_release,free_map_id,free_char_id,free_actor_id
0,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,...,NaN,1941-07-30,F,NaN,/m/03ttfc,Rosa Maria Sardà,54.0,/m/02vbt4w,NaN,/m/0gh6sw
1,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,...,NaN,1966,F,NaN,NaN,Mercè Pons,29.0,/m/02vb4j6,NaN,/m/0267qhz


In [76]:
# Making sure we haven't lost any information due to the merge (we used to as we performed inner merge before)
print(full_dataset['movie_name'].nunique())
print(full_dataset['movie_name'].isnull().sum())

75478
99


In [77]:
# We realize the outer merge has created an empty row with only the plot summary for each of the summaries 
# without corresponding ids, they are unexploitable => remove them
full_dataset = full_dataset.dropna(subset=['movie_name'])

In [78]:
full_dataset.tail()

,wiki_movie_id,free_movie_id_x,movie_name,release_x,box_office,runtime,languages,countries,genres,plot_summary,...,char_name,actor_birth,actor_gender,actor_height,actor_ethnicity,actor_name,age_at_release,free_map_id,free_char_id,free_actor_id
468075,37492363.0,/m/0ds7zbt,Cherries and Clover,2011.0,NaN,86.0,{},"{""/m/0d060g"": ""Canada""}","{""/m/05p553"": ""Comedy film"", ""/m/07s9rl0"": ""Dr...","When Clover's ' childhood friend, Cherries ', ...",...,NaN,NaN,NaN,NaN,NaN,Molly Cera,NaN,/m/0g4tzm6,NaN,/m/0g4tzm9
468076,37492363.0,/m/0ds7zbt,Cherries and Clover,2011.0,NaN,86.0,{},"{""/m/0d060g"": ""Canada""}","{""/m/05p553"": ""Comedy film"", ""/m/07s9rl0"": ""Dr...","When Clover's ' childhood friend, Cherries ', ...",...,NaN,NaN,NaN,NaN,NaN,Taylor Marie Milton,NaN,/m/0g4tzmk,NaN,/m/0g4tzmn
468077,37492363.0,/m/0ds7zbt,Cherries and Clover,2011.0,NaN,86.0,{},"{""/m/0d060g"": ""Canada""}","{""/m/05p553"": ""Comedy film"", ""/m/07s9rl0"": ""Dr...","When Clover's ' childhood friend, Cherries ', ...",...,NaN,NaN,NaN,NaN,NaN,Spencer Jenkins,NaN,/m/0g4tzmx,NaN,/m/0g4tzm_
468078,37501922.0,/m/0c0m5vt,Terminal Bliss,1992.0,NaN,91.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",Two adolescent children of wealthy parents dea...,...,John Hunter,1966-10-11,M,1.765,NaN,Luke Perry,25.0,/m/0gyqn_q,/m/0gyqn_s,/m/01g65g
468079,37501922.0,/m/0c0m5vt,Terminal Bliss,1992.0,NaN,91.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",Two adolescent children of wealthy parents dea...,...,Craig Murphy,1969-07-28,F,1.720,/m/041rx,Alexis Arquette,22.0,/m/0h35_7c,/m/0h35_7g,/m/02zjrf


In [79]:
full_dataset['actor_birth'] = full_dataset['actor_birth'].apply(extract_year)

In [80]:
# Drop the columns that were in both dataframes but not specified by lefton and righton in the merge function call
# Also drop ids that have become irrelevant
full_dataset.drop(columns=['free_movie_id_y', 'release_y'], inplace=True)

# Rename two columns in a DataFrame
full_dataset.rename(columns={'free_movie_id_x': 'free_movie_id', 'release_x': 'release'}, inplace=True)

In [81]:
full_dataset.head(5)

,wiki_movie_id,free_movie_id,movie_name,release,box_office,runtime,languages,countries,genres,plot_summary,char_name,actor_birth,actor_gender,actor_height,actor_ethnicity,actor_name,age_at_release,free_map_id,free_char_id,free_actor_id
0,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,NaN,1941.0,F,NaN,/m/03ttfc,Rosa Maria Sardà,54.0,/m/02vbt4w,NaN,/m/0gh6sw
1,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,NaN,1966.0,F,NaN,NaN,Mercè Pons,29.0,/m/02vb4j6,NaN,/m/0267qhz
2,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,NaN,1944.0,F,NaN,NaN,Anna Lizaran,51.0,/m/02vc7_7,NaN,/m/0263499
3,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,NaN,1935.0,F,NaN,NaN,Núria Espert,60.0,/m/02vbd74,NaN,/m/0263yvy
4,3217.0,/m/014hr,Army of Darkness,1992.0,21502796.0,81.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01q03"": ""Cult"", ""/m/03npn"": ""Horror"", ""/m...","After being pulled through a time portal, Ash ...",S-Mart Clerk,1965.0,M,1.87,NaN,Ted Raimi,26.0,/m/0hgcnkm,/m/0hgcnkq,/m/07qn0


In [82]:
full_dataset.isnull().sum()

wiki_movie_id           0
free_movie_id           0
movie_name              0
release             13205
box_office         366874
runtime             57941
languages               0
countries               0
genres                  0
plot_summary       155170
char_name          275286
actor_birth        123579
actor_gender        63020
actor_height       313256
actor_ethnicity    362022
actor_name          18639
age_at_release     175524
free_map_id         17411
free_char_id       275276
free_actor_id       18226
dtype: int64

In [83]:
# We see we can compute some of the NaNs in age at release due to having the birth year of the actor and the release date of the movie 
def compute_age_at_release(row):
    if np.isnan(row['age_at_release']) and pd.notna(row['release']) and pd.notna(row['actor_birth']):
        return row['release'] - row['actor_birth']
    return row['age_at_release']

# Apply the function to each row to fill missing 'age_at_release' values
full_dataset['age_at_release'] = full_dataset.apply(compute_age_at_release, axis=1)

In [84]:
full_dataset.isnull().sum()

wiki_movie_id           0
free_movie_id           0
movie_name              0
release             13205
box_office         366874
runtime             57941
languages               0
countries               0
genres                  0
plot_summary       155170
char_name          275286
actor_birth        123579
actor_gender        63020
actor_height       313256
actor_ethnicity    362022
actor_name          18639
age_at_release     132683
free_map_id         17411
free_char_id       275276
free_actor_id       18226
dtype: int64

In [85]:
# Creating a separate actor dataframe, since some of the actors information don't depend on the movie
rep_info = ['actor_name', 'free_actor_id', 'actor_birth', 'actor_gender', 'actor_ethnicity', 'actor_height']
actor = full_dataset[rep_info].drop_duplicates()

In [86]:
actor.head()

,actor_name,free_actor_id,actor_birth,actor_gender,actor_ethnicity,actor_height
0,Rosa Maria Sardà,/m/0gh6sw,1941.0,F,/m/03ttfc,NaN
1,Mercè Pons,/m/0267qhz,1966.0,F,NaN,NaN
2,Anna Lizaran,/m/0263499,1944.0,F,NaN,NaN
3,Núria Espert,/m/0263yvy,1935.0,F,NaN,NaN
4,Ted Raimi,/m/07qn0,1965.0,M,NaN,1.87


In [87]:
# Simplifying the final dataset by keeping a link between actor and itself via 'actor_name'
final_dataset = full_dataset.drop(columns=rep_info[1:])

In [88]:
final_dataset.head(4)

,wiki_movie_id,free_movie_id,movie_name,release,box_office,runtime,languages,countries,genres,plot_summary,char_name,actor_name,age_at_release,free_map_id,free_char_id
0,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,NaN,Rosa Maria Sardà,54.0,/m/02vbt4w,NaN
1,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,NaN,Mercè Pons,29.0,/m/02vb4j6,NaN
2,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,NaN,Anna Lizaran,51.0,/m/02vc7_7,NaN
3,330.0,/m/0ktn59,Actrius,1996.0,NaN,90.0,"{""/m/01m69"": ""Catalan language"", ""/m/06nm1"": ""...","{""/m/06mkj"": ""Spain""}","{""/m/07s9rl0"": ""Drama"", ""/m/01t_vv"": ""Comedy-d...",In order to prepare the role of an important o...,NaN,Núria Espert,60.0,/m/02vbd74,NaN


In [89]:
# Creating a separate movies_info dataframe, since some of the movie information don't vary for a single same movie
rep_info = ['movie_name', 'wiki_movie_id', 'free_movie_id', 'release', 'box_office', 'runtime', 'languages', 'countries', 'genres', 'plot_summary']
movies_info = full_dataset[rep_info].drop_duplicates()

In [90]:
# Simplifying the final dataset by keeping a link between movies_info and itself via 'movie_name'
final_dataset = final_dataset.drop(columns=rep_info[1:])

In [91]:
final_dataset.head()

,movie_name,char_name,actor_name,age_at_release,free_map_id,free_char_id
0,Actrius,NaN,Rosa Maria Sardà,54.0,/m/02vbt4w,NaN
1,Actrius,NaN,Mercè Pons,29.0,/m/02vb4j6,NaN
2,Actrius,NaN,Anna Lizaran,51.0,/m/02vc7_7,NaN
3,Actrius,NaN,Núria Espert,60.0,/m/02vbd74,NaN
4,Army of Darkness,S-Mart Clerk,Ted Raimi,26.0,/m/0hgcnkm,/m/0hgcnkq


In [92]:
# Set hierarchical indexing with 'movie_name' as the primary index and character-specific columns as secondary
final_dataset.set_index(['movie_name', 'char_name', 'actor_name'], inplace=True)

# Display the DataFrame with the hierarchical index
final_dataset.head()

age_at_release free_map_id  \
movie_name       char_name    actor_name                                     
Actrius          NaN          Rosa Maria Sardà            54.0  /m/02vbt4w   
                              Mercè Pons                  29.0  /m/02vb4j6   
                              Anna Lizaran                51.0  /m/02vc7_7   
                              Núria Espert                60.0  /m/02vbd74   
Army of Darkness S-Mart Clerk Ted Raimi                   26.0  /m/0hgcnkm   

                                               free_char_id  
movie_name       char_name    actor_name                     
Actrius          NaN          Rosa Maria Sardà          NaN  
                              Mercè Pons                NaN  
                              Anna Lizaran              NaN  
                              Núria Espert              NaN  
Army of Darkness S-Mart Clerk Ted Raimi          /m/0hgcnkq

In [93]:
# Let us remove the ids for languages countries and genres as they are no longer needed
import ast

def extract_values_if_str_dict(value):
    """Parses string as a dictionary and extracts values if possible, otherwise returns the original value."""
    try:
        # Attempt to parse the string as a dictionary
        parsed_value = ast.literal_eval(value)
        if isinstance(parsed_value, dict):
            return list(parsed_value.values())
    except (ValueError, SyntaxError):
        # Return the original value if parsing fails
        return value

# Apply the function to each relevant column
movies_info['languages'] = movies_info['languages'].apply(extract_values_if_str_dict)
movies_info['countries'] = movies_info['countries'].apply(extract_values_if_str_dict)
movies_info['genres'] = movies_info['genres'].apply(extract_values_if_str_dict)

# Display the modified DataFrame
movies_info.head()

,movie_name,wiki_movie_id,free_movie_id,release,box_office,runtime,languages,countries,genres,plot_summary
0,Actrius,330.0,/m/0ktn59,1996.0,NaN,90.0,"[Catalan language, Spanish Language]",[Spain],"[Drama, Comedy-drama]",In order to prepare the role of an important o...
4,Army of Darkness,3217.0,/m/014hr,1992.0,21502796.0,81.0,[English Language],[United States of America],"[Cult, Horror, Stop motion, Costume drama, Act...","After being pulled through a time portal, Ash ..."
18,The Birth of a Nation,3333.0,/m/0151l,1915.0,50000000.0,190.0,"[Silent film, English Language]",[United States of America],"[Silent film, Indie, Costume drama, Epic, Blac...",The film follows two juxtaposed families: the...
32,Blade Runner,3746.0,/m/017n9,1982.0,33139618.0,116.0,"[Japanese Language, Cantonese, English Languag...","[United States of America, Hong Kong]","[Thriller, Cyberpunk, Science Fiction, Future ...","{{Hatnote}} In Los Angeles, November 2019, ret..."
47,Blazing Saddles,3837.0,/m/018f8,1974.0,119500000.0,93.0,"[Yiddish Language, English Language]",[United States of America],"[Western, Satire, Comedy]","In the American Old West of 1874, construction..."


In [94]:
# We will map the ethnicity id to its corresponding value
unique_ethnicities = actor['actor_ethnicity'].unique()

# Function to split the list into batches
def split_into_batches(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Create an empty dictionary to store the mappings
freebase_to_wikidata_mapping = {}

# Iterate over batches of Freebase IDs
batch_size = 50  # Set batch size to 50 to avoid long URL issues
for batch in split_into_batches(unique_ethnicities, batch_size):
    # Create a batch SPARQL query
    query = """
    SELECT ?freebase_id ?item ?itemLabel WHERE {
      VALUES ?freebase_id {""" + " ".join([f'"{fb_id}"' for fb_id in batch]) + """}
      ?item wdt:P646 ?freebase_id.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    # Endpoint for Wikidata SPARQL
    url = "https://query.wikidata.org/sparql"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json"
    }

    # Send the request to Wikidata
    response = requests.get(url, headers=headers, params={"query": query, "format": "json"})

    # Check the status and response
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and 'bindings' in data['results']:
            for result in data['results']['bindings']:
                freebase_id = result['freebase_id']['value']
                wikidata_id = result['item']['value'].split('/')[-1]
                label = result['itemLabel']['value']
                freebase_to_wikidata_mapping[freebase_id] = {
                    "wikidata_id": wikidata_id,
                    "label": label
                }
        else:
            print("No valid data found in response for this batch.")
    else:
        print(f"Error: Received status code {response.status_code} for batch starting with {batch[0]}")

# Assign labels for ethnicity using the batch lookup dictionary
actor['actor_ethnicity_label'] = actor['actor_ethnicity'].map(
    lambda x: freebase_to_wikidata_mapping.get(x, {}).get('label', 'Unknown')
)

In [95]:
males = actor['actor_gender'][actor['actor_gender']=='M'].count()
females = actor['actor_gender'][actor['actor_gender']=='F'].count()
print(males, females, actor['actor_gender'].isna().sum())
total = males + females
male_perc = males/total
female_perc = females/total

print(actor['actor_gender'].isnull().sum())


actor['actor_gender'] = actor['actor_gender'].fillna(
    lambda: np.random.choice(['M', 'F'], p=[male_perc, female_perc])
)



print(actor['actor_gender'].isnull().sum())

61519 35864 38378
38378
0


In [96]:
actor.head()

,actor_name,free_actor_id,actor_birth,actor_gender,actor_ethnicity,actor_height,actor_ethnicity_label
0,Rosa Maria Sardà,/m/0gh6sw,1941.0,F,/m/03ttfc,NaN,Spaniards
1,Mercè Pons,/m/0267qhz,1966.0,F,NaN,NaN,Unknown
2,Anna Lizaran,/m/0263499,1944.0,F,NaN,NaN,Unknown
3,Núria Espert,/m/0263yvy,1935.0,F,NaN,NaN,Unknown
4,Ted Raimi,/m/07qn0,1965.0,M,NaN,1.87,Unknown


In [97]:
print(actor['actor_name'].isnull().sum())

actor['actor_name'] = actor['actor_name'].fillna(
    lambda: "Unknown"
)


print(actor['actor_name'].isnull().sum())

339
0


In [98]:
actor.head()

,actor_name,free_actor_id,actor_birth,actor_gender,actor_ethnicity,actor_height,actor_ethnicity_label
0,Rosa Maria Sardà,/m/0gh6sw,1941.0,F,/m/03ttfc,NaN,Spaniards
1,Mercè Pons,/m/0267qhz,1966.0,F,NaN,NaN,Unknown
2,Anna Lizaran,/m/0263499,1944.0,F,NaN,NaN,Unknown
3,Núria Espert,/m/0263yvy,1935.0,F,NaN,NaN,Unknown
4,Ted Raimi,/m/07qn0,1965.0,M,NaN,1.87,Unknown


In [99]:
movies_info.head()

,movie_name,wiki_movie_id,free_movie_id,release,box_office,runtime,languages,countries,genres,plot_summary
0,Actrius,330.0,/m/0ktn59,1996.0,NaN,90.0,"[Catalan language, Spanish Language]",[Spain],"[Drama, Comedy-drama]",In order to prepare the role of an important o...
4,Army of Darkness,3217.0,/m/014hr,1992.0,21502796.0,81.0,[English Language],[United States of America],"[Cult, Horror, Stop motion, Costume drama, Act...","After being pulled through a time portal, Ash ..."
18,The Birth of a Nation,3333.0,/m/0151l,1915.0,50000000.0,190.0,"[Silent film, English Language]",[United States of America],"[Silent film, Indie, Costume drama, Epic, Blac...",The film follows two juxtaposed families: the...
32,Blade Runner,3746.0,/m/017n9,1982.0,33139618.0,116.0,"[Japanese Language, Cantonese, English Languag...","[United States of America, Hong Kong]","[Thriller, Cyberpunk, Science Fiction, Future ...","{{Hatnote}} In Los Angeles, November 2019, ret..."
47,Blazing Saddles,3837.0,/m/018f8,1974.0,119500000.0,93.0,"[Yiddish Language, English Language]",[United States of America],"[Western, Satire, Comedy]","In the American Old West of 1874, construction..."


In [100]:
final_dataset.head(5)

age_at_release free_map_id  \
movie_name       char_name    actor_name                                     
Actrius          NaN          Rosa Maria Sardà            54.0  /m/02vbt4w   
                              Mercè Pons                  29.0  /m/02vb4j6   
                              Anna Lizaran                51.0  /m/02vc7_7   
                              Núria Espert                60.0  /m/02vbd74   
Army of Darkness S-Mart Clerk Ted Raimi                   26.0  /m/0hgcnkm   

                                               free_char_id  
movie_name       char_name    actor_name                     
Actrius          NaN          Rosa Maria Sardà          NaN  
                              Mercè Pons                NaN  
                              Anna Lizaran              NaN  
                              Núria Espert              NaN  
Army of Darkness S-Mart Clerk Ted Raimi          /m/0hgcnkq

In [101]:
useful_actors = actor[['actor_name', 'actor_gender', 'actor_ethnicity_label', 'actor_height']]
useful_movies = movies_info[['movie_name', 'release', 'box_office', 'runtime', 'languages', 'countries', 'genres', 'plot_summary']]
useful_final = final_dataset.reset_index()[['movie_name', 'actor_name', 'age_at_release']]

merge1 = useful_final.merge(useful_actors, on='actor_name', how='inner')

final_merged = merge1.merge(useful_movies, on='movie_name', how='inner')

print(useful_actors.shape, useful_movies.shape, useful_final.shape)
print(final_merged.shape)

(135761, 4) (81741, 8) (468080, 3)
(592933, 13)


In [102]:
import ast

# Convert the 'countries' column from a string representation to actual lists
final_merged['countries'] = final_merged['countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
final_merged['languages'] = final_merged['languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
final_merged['genres'] = final_merged['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [103]:
# Extracting the subdatasets based on the clusters defined below in order to make sure the global distr makes sense
oceanic_cluster = ["Australia", "New-Zealand"]
NA_cluster = ["United States of America", "Canada"]
Indian_cluster = ["India"]
East_Asian_Cluster = ["Japan", "China", "South Korea", "Hong-Kong", "Taïwan"]
European_Cluster = ["France", "Spain", "Italy", "united Kingdom"]


# Get a boolean mask for rows where the condition is met, handling non-iterable types
oceanic_mask = final_merged['countries'].apply(
    lambda x: any(country in oceanic_cluster for country in x) if isinstance(x, (list, np.ndarray)) else x
)

NA_mask = final_merged['countries'].apply(
    lambda x: any(country in NA_cluster for country in x) if isinstance(x, (list, np.ndarray)) else x
)

Indian_mask = final_merged['countries'].apply(
    lambda x: any(country in Indian_cluster for country in x) if isinstance(x, (list, np.ndarray)) else x
)

East_Asian_mask = final_merged['countries'].apply(
    lambda x: any(country in East_Asian_Cluster for country in x) if isinstance(x, (list, np.ndarray)) else x
)

European_Mask = final_merged['countries'].apply(
    lambda x: any(country in European_Cluster for country in x) if isinstance(x, (list, np.ndarray)) else x
)

# Get the integer locations of these rows
oceanic_ilocs = np.where(oceanic_mask)[0]
NA_ilocs = np.where(NA_mask)[0]
Indian_ilocs = np.where(Indian_mask)[0]
East_Asian_ilocs = np.where(East_Asian_mask)[0]
European_ilocs = np.where(European_Mask)[0]

oceanic_main = final_merged.iloc[oceanic_ilocs]
NA_main = final_merged.iloc[NA_ilocs]
Indian_main = final_merged.iloc[Indian_ilocs]
East_Asian_main = final_merged.iloc[East_Asian_ilocs]
European_main = final_merged.iloc[European_ilocs]

In [104]:
actor.isnull().sum()

actor_name                    0
free_actor_id                 1
actor_birth               77202
actor_gender                  0
actor_ethnicity          127610
actor_height             122905
actor_ethnicity_label         0
dtype: int64

We see that we are still missing a lot of ethnicities, birthdates and gender which is inherently due to our database, we will try later to remediate to this by using scrapping

### III) Saving the new cleaned Datasets

In [105]:
import os

# Specify the folder name
folder_names = ["cleaned_datasets", "clustered_clean"]


# Create the folder if it doesn't already exist
for folder_name in folder_names:
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Folder '{folder_name}' already exists.")


Folder 'cleaned_datasets' already exists.
Folder 'clustered_clean' already exists.


In [106]:
# List of DataFrames and their respective filenames

flat_df = final_dataset.reset_index()

dataframes = {
    "movies_info.csv": movies_info,
    "main_df.csv": flat_df,
    "actor.csv": actor
}

dataframes_clustered = {
    "oceanic_df.csv": oceanic_main,
    "NA_df.csv": NA_main,
    "Indian_df.csv": Indian_main,
    "East_asian_df.csv": East_Asian_main,
    "Europe_df.csv": European_main
}

dfs = [dataframes, dataframes_clustered]

count = 0
for df in dfs:
    # Save each DataFrame to a CSV file in the new folder
    for filename, df in df.items():
        file_path = os.path.join(folder_names[count], filename)
        df.to_csv(file_path, index=False)
        print(f"DataFrame saved to {file_path}")
    count +=1


DataFrame saved to cleaned_datasets/movies_info.csv
DataFrame saved to cleaned_datasets/main_df.csv
DataFrame saved to cleaned_datasets/actor.csv
DataFrame saved to clustered_clean/oceanic_df.csv
DataFrame saved to clustered_clean/NA_df.csv
DataFrame saved to clustered_clean/Indian_df.csv
DataFrame saved to clustered_clean/East_asian_df.csv
DataFrame saved to clustered_clean/Europe_df.csv


In [107]:
print(movies_info.shape)

(81741, 10)
